In [1]:
from faster_config import Config
from faster_data_source import DataSource
from faster_file_manager import FileManager
from faster_metrics import MetricsAggregator
from faster_model_evaluator import ModelEvaluator
from faster_trainer import ModelTrainer
from faster_plotter import Plotter
from faster_model_loader import ModelLoader

In [2]:
Config.set_local_settings()
file_manager = FileManager()
data_source = DataSource(file_manager)
model_loader = ModelLoader(file_manager)
model_trainer = ModelTrainer(file_manager)
metrics_aggregator = MetricsAggregator(file_manager)
plotter = Plotter(file_manager, metrics_aggregator)

 Loading Annotations:  InBreast-COCO\train\_annotations.coco.json 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
model = model_loader.new_pretrained_model()

data_source.start_kfold()
datasets, dataloaders = data_source.testing()
train_dataset, valid_dataset = datasets
train_loader, valid_loader = dataloaders


 New Model:  Faster-RCNN 
 Loading Annotations:  InBreast-COCO\test\_annotations.coco.json 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
train_dataset.__getitem__(1)[1]

{'boxes': tensor([[0.7284, 0.8652, 0.8557, 1.0000]]),
 'labels': tensor([0]),
 'area': tensor([10978.2715]),
 'iscrowd': tensor([0.]),
 'image_id': tensor([1])}

In [5]:
batch_train = next(iter(train_loader))
batch_valid = next(iter(valid_loader))

In [4]:
# Validation - Threshold Optimization

best_model = model_loader.load_best_model()
model_evaluator = ModelEvaluator(best_model, plotter)

threshold = 0.001
valid_metrics = model_evaluator.evaluate(valid_dataset, valid_loader, threshold, save_plots=True)
valid_metrics

 Loading Model:  lightning_logs\InBreast-COCO\Faster-RCNN\test\checkpoints\epoch=0-step=1.ckpt 
 Evaluating on threshold:  0.001 


{'Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.0}